# Mounting Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Imports

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

In [ ]:
!pip install tf_slim

In [ ]:
!pip install pycocotools

# Setting Environment Variables on colab!

In [ ]:
cd drive/My\ Drive/Tensorflow/models/research/

/content/drive/My Drive/Tensorflow/models/research


In [ ]:
import os
path = os.getcwd()
path

'/content/drive/My Drive/Tensorflow/models/research'

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.
os.environ['PYTHONPATH'] +=  ":" + path + ":" + path +"/slim"


object_detection/protos/input_reader.proto: warning: Import object_detection/protos/image_resizer.proto but not used.


In [ ]:
!echo $PYTHONPATH

/tensorflow-1.15.2/python3.6:/env/python:/content/drive/My Drive/Tensorflow/models/research:/content/drive/My Drive/Tensorflow/models/research/slim


In [ ]:
import sys
sys.path.insert(1, path+"/slim")
sys.path.insert(1, path)

In [ ]:
# testing the model builder
!python3 object_detection/builders/model_builder_test.py

python3: can't open file 'object_detection/builders/model_builder_test.py': [Errno 2] No such file or directory


# Training the Model  

In [ ]:
#path to object detection folder!
PATH ="../models/research/object_detection/"

In [ ]:
cd ../../License\ Plate\ Detection

/content/drive/My Drive/Tensorflow/License Plate Detection


In [ ]:
!python {PATH}model_main.py --alsologtostderr --model_dir=training/ --pipeline_config_path=Data/ssd_mobilenet_v1_pets.config

# Visualising results on Tensorboard

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
#not necessary
!tensorboard  --logdir=training --inspect

In [ ]:
%tensorboard --logdir training

# Exporting Frozen inference graph

## the normal version(using export_inference_graph.py )

In [ ]:
!python {PATH}export_inference_graph.py --input_type image_tensor --pipeline_config_path Data/ssd_mobilenet_v1_pets.config --trained_checkpoint_prefix training/model.ckpt-32375 --output_directory output_folder/output_inference_graph.pb

## tflite version (using  export_tflite_ssd_graph.py)

In [ ]:
!python {PATH}export_tflite_ssd_graph.py --input_type image_tensor --pipeline_config_path Data/ssd_mobilenet_v1_pets.config --trained_checkpoint_prefix training/model.ckpt-32375 --output_directory output_folder/tflite_output_inference_graph.pb --add_postprocessing_op=true

In [ ]:
cd output_folder/tflite_output_inference_graph.pb

In [ ]:
!tflite_convert \
    --output_file=detect.tflite \
    --graph_def_file=tflite_graph.pb \
    --input_shapes=1,300,300,3 \
    --input_arrays=normalized_input_image_tensor \
    --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'  \
    --inference_type=QUANTIZED_UINT8 \
    --mean_values=128 \
    --std_dev_values=128 \
    --change_concat_input_ranges=false \
    --allow_custom_ops

In [ ]:
cd ../../

# Evaluating on test images again with the final model

In [ ]:
!python model_main.py --pipeline_config_path=Data/ssd_mobilenet_v1_pets.config --model_dir=training --checkpoint_dir=output_folder/output_inference_graph_v1.pb --run_once=True